In [ ]:
# CATEGORY

In [ ]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
# display(melb_data.head())
melb_df = melb_data.copy()
# display(melb_df.head())
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
#melb_df.head()
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea']/total_rooms
#display(melb_df['MeanRoomsSquare'])
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])
price_square = melb_df['Price']**2
#display(price_square)

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# print(melb_df['Date'])

year_sold = melb_df['Date'].dt.year
#print('год продажи', year_sold)
#print('min yearsold', year_sold.min())
#print('max yearsold', year_sold.max())
#print('mode of sold', year_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop(['YearBuilt'], axis=1)
#display(melb_df.head())

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
#print(weekend_count, 'объектов недвижимости было продано в выходные')

# В наших данных есть столбец с адресами объектов недвижимости
# check количество уникальных значений в столбце с помощью метода nunique()
#print('количество уникальных значений в столбце', melb_df['Address'].nunique())

# давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address
#print(melb_df['Address'].loc[177])
#print(melb_df['Address'].loc[1812])
#print(melb_df['Address'].loc[9001])

# Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E'] # Создаём список географических пометок exclude_list.
    address_list = address.split(' ') # Метод split() разбивает строку на слова по пробелу, получаем список слов в строке 
    street_type = address_list[-1] # Обрезаем список, оставляя в нём только последний элемент, потенциальный подтип улицы
    if street_type in exclude_list: # Делаем проверку на то, что полученный подтип является географической пометкой at exclude_list
        street_type = address_list[-2] # Если переменная street_type является географической пометкой change на второй элемент с конца 
    return street_type
# Теперь применим эту функцию к столбцу c адресом 
# передадим функцию get_street_type в аргумент метода столбца apply()
street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
# функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу

# мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось
#print(street_types.nunique())

popular_stypes = street_types.value_counts().nlargest(10).index
#print(popular_stypes)

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'others')
#display(melb_df['StreetType'])

melb_df = melb_df.drop('Address', axis=1)
#display(melb_df.head())

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
#print(round(melb_df[(melb_df['Weekend'] == 1)]['Price'].mean()))